In [0]:
fname = '/Users/klay6683/Dropbox/DDocuments/planet4/hireport_south_mars_zoo_v6.xls'

#South Year 1

In [0]:
df = pd.read_excel(fname, sheet='South_year_1')

In [0]:
# df.dropna(how='all', inplace=True)

In [0]:
#df.time = pd.to_datetime(df.time)

In [0]:
df.head()

In [0]:
df.dtypes

In [0]:
def repair_time(tstr):
    if type(tstr) == float:
        return tstr
    date, time = tstr.split('T')
    date_tokens = date.split('-')
    newdate = '-'.join(date_tokens[:3])
    return "{}T{}".format(newdate, time).split('.')[0]

status = pd.read_hdf('image_name_status.h5','df')
def fix_and_merge(df):
    df.time = df.time.map(repair_time)    
    df.columns = ['obs_id','Lat','Lon','Ls', 'time', 'Comment','Unnamed1','Unnamed2']
    oldindex = df.index
    merged = df.merge(status, left_on='obs_id', right_index=True).reindex(oldindex)
    merged = merged.drop(['Unnamed1','Unnamed2'], axis=1)
    return merged

In [0]:
df.time = df.time.map(repair_time)

In [0]:
try:
    df.time = pd.to_datetime(df.time, format='%Y-%b-%dT%H:%M:%S', errors='raise')
except ValueError as e:
    print(e)

In [0]:
df.head()

In [0]:
status.columns

In [0]:
merged.columns

In [0]:
merged = merged.drop(

In [0]:
merged.to_excel('merged_with_status_excel_sheet.xlsx')

In [0]:
# %load /Users/klay6683/Downloads/spstere.py
from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt
# setup north polar stereographic basemap.
# The longitude lon_0 is at 6-o'clock, and the
# latitude circle boundinglat is tangent to the edge  
# of the map at lon_0. Default value of lat_ts
# (latitude of true scale) is pole.
m = Basemap(projection='spstere',boundinglat=-80,lon_0=90,resolution='l')
m.drawcoastlines()
m.fillcontinents(color='coral',lake_color='aqua')
# draw parallels and meridians.
m.drawparallels(np.arange(-80.,81.,20.))
m.drawmeridians(np.arange(-180.,181.,20.))
m.drawmapboundary(fill_color='aqua') 
# draw tissot's indicatrix to show distortion.
ax = plt.gca()
for y in np.linspace(19*m.ymin/20,m.ymin/20,10):
    for x in np.linspace(19*m.xmin/20,m.xmin/20,10):
        lon, lat = m(x,y,inverse=True)
        poly = m.tissot(lon,lat,2.5,100,\
                        facecolor='green',zorder=10,alpha=0.5)
plt.title("South Polar Stereographic Projection")
plt.show()


In [0]:
%matplotlib inline

In [0]:
from mpl_toolkits.basemap import Basemap
import matplotlib as mpl

class PoleMapper(object):
    """docstring for PoleMapper"""

#     palette = mpl.cm.cubehelix
    
    def __init__(self, blat, gridpts=1000, round=True):
        super(PoleMapper, self).__init__()
        self.blat = blat
        self.gridpts = gridpts
        self.basemap = Basemap(lon_0=180, boundinglat=blat, 
                         projection='spstere', round=round)
        self.pole = 'Southpole' if blat < 0 else 'Northpole'
        
    def create_map(self, hdata, strings, ax=None, vmin=None, vmax=None):
        mode, chid, dayside = strings
        if not ax:
            fig, ax = plt.subplots()
        self.palette.set_bad(ax.get_axis_bgcolor(), 1.0)
        CS = self.basemap.pcolormesh(hdata.xedges, hdata.yedges, hdata.H.T,
                               shading='flat', cmap=self.palette,
                               ax = ax, vmin=vmin, vmax=vmax)
        self.basemap.drawparallels(np.arange(-90, self.blat),latmax=-90, ax=ax, 
                                    labels=[1,1,1,1])
        self.basemap.drawmeridians(np.arange(0,360,30),latmax=-90, ax=ax,
                                    labels=[1,1,1,1])
        self.basemap.colorbar(CS,ax=ax)
        ax.set_title(' '.join([self.pole, chid, mode, dayside]))
        plt.savefig('_'.join(['southpole', chid, mode, str(self.gridpts),
                              dayside]) + '.png', dpi=300)

    def create_scatter_map(self, lon, lat, ax=None):
        if ax is None:
            fig, ax = plt.subplots()
        CS = self.basemap.scatter(lon, lat, latlon=True)
        self.basemap.drawparallels(np.arange(-90, self.blat, 5), latmax=90, ax=ax)
        meridians = self.basemap.drawmeridians(np.arange(0, 360, 30), labelstyle='+/-', latmax=90, ax=ax,
                                   labels=[1, 1, 1, 1],
                                  )
        ax.set_title('Southpole of Mars')
        return meridians
    
    def create_multimap(self, data, strings):
        n = len(data)
        fig, ax = plt.subplots(1, n)
        for i,d in enumerate(data):
            self.create_map(d, strings[i], ax.flatten()[i])


In [0]:
fig, ax = plt.subplots(figsize=(8,8))
pm = PoleMapper(-65, round=True)
meridians = pm.create_scatter_map(df.Lon.values, df.Lat.values, ax=ax)

In [0]:
for i in meridians.items():
    to = i[1][1][0]

In [0]:
print(to)

In [0]:
df[df.Lat==df.Lat.max()]

# South year 2

In [0]:
fname = fname+'x'

In [0]:
fname

In [0]:
excelfile = pd.ExcelFile(fname)

In [0]:
df = excelfile.parse(1)

In [0]:
df.head()

In [0]:
df.time = df.time.map(repair_time)
df = df.loc[:, df.columns[:-7]]
oldindex = df.index
merged = df.merge(status, left_on='Observation_id', right_index=True).reindex(oldindex)
merged.head()

In [0]:
merged.to_excel('merged_season2.xlsx')

In [0]:
df = excelfile.parse(2)
df.head()

In [0]:
df = df.loc[:, df.columns[:7]]

In [0]:
df.head()

In [0]:
df.time = df.time.map(repair_time)
oldindex = df.index
merged = df.merge(status, left_on='Observation_id', right_index=True).reindex(oldindex)
merged.head()

In [0]:
merged.to_excel('merged_season3.xlsx')

In [0]:
df = excelfile.parse(3)
df.head()

In [0]:
df = df.loc[:, df.columns[:-2]]
df.head()

In [0]:
df.time = df.time.map(repair_time)
oldindex = df.index
merged = df.merge(status, left_on='observation_id', right_index=True).reindex(oldindex)
merged.head()

In [0]:
merged.to_excel('merged_season4.xlsx')